In [1]:
pip install praw

     |████████████████████████████████| 153kB 3.4MB/s 
     |████████████████████████████████| 204kB 10.8MB/s 


In [2]:
import praw
import pandas as pd
# Reddit credentials, password stored in .env 
# PRAW setup and login goes here, omitted for privacy

In [3]:
# set up some sort of corpus of keywords to snag specific reddit entries
all_locs = pd.read_json('https://raw.githubusercontent.com/2020PB/police-brutality/data_build/all-locations-v2.json')
all_locs = pd.json_normalize(all_locs['data'])
all_locs = all_locs.drop(columns=['edit_at','id'])
def cleanlinks(json):
    links_out = []
    for link in json:
        links_out.append(link['url'])
    return links_out
all_locs['links'] = all_locs['links'].apply(cleanlinks)
all_locs['date'] = pd.to_datetime(all_locs['date'],format='%Y-%m-%d')
all_tags = all_locs['tags'].copy()
tags = set()
for taglist in all_tags:
  for tag in taglist:
    if tag not in tags:
      tags.add(tag)
# manually remove needless tags
print(tags)
tags.discard('')
tags.discard('medic')
tags.discard('bike')
tags.discard('non-protest')
tags.discard('pregnant')
tags.discard('lgbtq+')
tags.discard('racial-profiling')
tags.discard('legal-observer')
tags.discard('tear-gas-canister')
tags.discard('politician')
tags.discard('incitement')
tags.discard('homeless')
tags.discard('shoot')
tags.discard('strike')
tags.discard('elderly')
tags.discard('vehicle')
tags.discard('inhumane-treatment')
tags.discard('journalist')
tags.discard('throw')
tags.discard('explosive')
tags.discard('threaten')
tags.discard('horse')
tags.discard('shove')
tags.discard('child')
tags.discard('shield')
tags.discard('dog')
tags.discard('knee')
tags.discard('protester')
tags.discard('gun')
tags.discard('conceal')
tags.discard('bystander')
tags.discard('grab')
tags.discard('push')
tags.discard('zip-tie')
tags.discard('spray')
tags.discard('drive')
tags.discard('person-with-disability')
tags.discard('celebrity')
tags.discard('projectile')
tags.discard('beat')
newtags = set()
for tag in tags:
  tag = newtags.add(tag.replace('-',' '))
tags = newtags
print(tags)

{'', 'shove', 'shoot', 'legal-observer', 'push', 'gun', 'vehicle', 'knee-on-neck', 'explosive', 'foam-bullet', 'death', 'tear-gas', 'body-cam', 'drive', 'politician', 'marking-round', 'medic', 'spray', 'pregnant', 'tear-gas-canister', 'grab', 'kick', 'person-with-disability', 'bystander', 'knee', 'protester', 'tackle', 'rubber-bullet', 'mace', 'headlock', 'incitement', 'punch', 'journalist', 'wooden-bullet', 'taser', 'zip-tie', 'conceal', 'live-round', 'dog', 'shield', 'arrest', 'hide-badge', 'non-protest', 'pepper-spray', 'celebrity', 'inhumane-treatment', 'elderly', 'threaten', 'racial-profiling', 'bean-bag', 'stun-grenade', 'lrad', 'pepper-ball', 'choke', 'baton', 'property-destruction', 'bike', 'projectile', 'throw', 'abuse-of-power', 'child', 'tase', 'lgbtq+', 'gas', 'beat', 'homeless', 'horse', 'strike'}
{'pepper ball', 'stun grenade', 'property destruction', 'knee on neck', 'pepper spray', 'kick', 'marking round', 'tackle', 'tear gas', 'mace', 'headlock', 'punch', 'body cam', 'l

In [4]:
# Grabbing 100 hottest posts on Reddit at the moment. Will filter for police use of force later

data = []

# other possible subreddits: publicfreakout, allcopnodonut
for submission in reddit.subreddit("news").hot(limit=1000):
  data.append([submission.id, submission.title, submission.score, submission.subreddit, submission.url, 
               submission.num_comments, submission.selftext, submission.created])

# We'll need a way to get coordinates for a given post, before we include that in df
col_names = ['id', 'title', 'score', 'subreddit', 'url', 
             'num_comments', 'text', 'created']
df = pd.DataFrame(data, columns=col_names)

df.head()

,id,title,score,subreddit,url,num_comments,text,created
0,idwafa,Man dies in Arizona after being restrained by ...,85584,news,https://news.sky.com/story/man-dies-in-arizona...,10502,,1.598044e+09
1,idrvob,Belarusian museum director who refused to sign...,44192,news,http://khpg.org/en/index.php?id=1597789267&fbc...,1112,,1.598021e+09
2,idtpyq,"For the first time in more than 100 years, the...",4223,news,https://www.cnn.com/2020/08/20/us/wolverines-r...,148,,1.598032e+09
3,ie14w6,Activists find camera inside mysterious box on...,970,news,https://www.fox13memphis.com/news/local/activi...,175,,1.598060e+09
4,idrwp2,Leaked photos show disassembled USPS machines ...,6625,news,https://nbcmontana.com/news/local/leaked-photo...,791,,1.598022e+09


In [5]:
import requests
from bs4 import BeautifulSoup
import re
# get the url of the reddit post
for url in df['url']:
  # get the HTML from the url
  try:
    r = requests.get(url, timeout=10)
  except:
    continue
  soup = BeautifulSoup(r.text)
  # get tags from metadata for the site
  sitetags = set()
  for meta in soup.find_all('meta'):
    if meta is None:
      continue
    meta = str(meta)
    meta = meta.lower()
    meta = re.sub('[\W_]+',' ', meta)
    for tag in str(meta).split():
      sitetags.add(tag)
  tags_final = sitetags & tags
  if tags_final:
    # some matches found, print title and matched tags
    print(soup.title.text.strip(), tags_final)

Steve Bannon pleads not guilty to fraud charges in border wall scheme - CBS News {'arrest'}
Driver who said God spoke to him before fatal crash found not guilty by reason of insanity | Courts | theadvocate.com {'baton'}
Footage of black Muslim man who died in police custody renews calls for justice | US news | The Guardian {'death'}
Curt Schilling says he hasn't been arrested alongside Steve Bannon {'arrest'}
Suspect arrested after shooting at Vons gas station in Nipomo | Crime and Courts | santamariatimes.com {'gas'}
Loughlin, Giannulli get prison time in college bribery plot {'arrest'}
4 people missing after pipeline explosion at port in Texas {'gas'}
Barr: Feds to appeal ruling, seek death for Boston bomber {'death'}
New York couple arrested for allegedly harassing neighbor after 3-year back and forth - CNN {'arrest'}
CPD: Father watches wife's murder on doorbell camera; Son charged in death | WTVC {'death'}
UConn students evicted from dorms for holding pandemic party as schools gra

In [16]:
# Test grabbing the contents of the articles themselves to imporve spaCy NLP
# get the url of the reddit post
import re
for url in df['url'][:10]:
  # get the HTML from the url
  try:
    r = requests.get(url, timeout=10)
  except:
    continue
  soup = BeautifulSoup(r.text)
  # get text from website
  output_text = " ".join([x.text for x in soup.find_all('p')])
  output_text = re.sub('[^a-zA-Z0-9.,\']+', ' ', output_text).strip()
  print(output_text)

Friday 21 August 2020 12 21, UK Police in Arizona have released bodycam and CCTV footage showing the arrest of a man who died after he was held by officers on hot tarmac for six minutes in 100 degree heat. Ramon Timothy Lopez, 28, became unresponsive in the back of a police car following his arrest for allegedly stealing a drink from a shop in the city of Phoenix. The cause of death is pending from the medical examiner, while two separate investigations are under way into the incident and the actions of the police officers involved. Lopez's brother David Gonzalez told local station KNXV TV Nobody's perfect, but he didn't deserve any of that. He added They were on top of him with all their weight on the hot asphalt. Who wouldn't be fighting He was fighting for his life, and he lost. The incident took place on 4 August after police received a call from a woman reporting a man behaving suspiciously, making obscene gestures and looking at people's cars. He's kind of acting funny, he's over